In [ ]:
m

# Example of ColossalAI applied on an external dataset


Creating the dataset

Kindly download the train.zip from https://www.kaggle.com/c/dogs-vs-cats/data

And create Train directory in Colab Notebooks directory
or import dataset and change code accordingly


In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['Colab Notebooks']

unzipping train.zip


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/train.zip" -d "/content/drive/MyDrive/Colab Notebooks/Train/"

In [3]:
import pandas as pd
import os
import torch

train_df = pd.DataFrame(columns=["img_name","label"])
train_df["img_name"] = os.listdir("/content/drive/MyDrive/Colab Notebooks/Train/train/")
for idx, i in enumerate(os.listdir("/content/drive/MyDrive/Colab Notebooks/Train/train/")):
    if "cat" in i:
        train_df["label"][idx] = 0
    if "dog" in i:
        train_df["label"][idx] = 1

train_df.to_csv (r'train_csv.csv', index = False, header=True)

created train.csv containg the image title along with label

In [4]:
import numpy as np

In [5]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class CatsAndDogsDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))
       
        
        if self.transform is not None:
            img = self.transform(img)
        # print(img.dtype)
        # print(y_label.dtype)
        img = np.asarray(np.copy(img), dtype='float32')
        label = np.asarray(np.copy(y_label), dtype='float32')
        img = np.expand_dims(img, axis=0)
        # Load the data into PyTorch tensors
        img = torch.from_numpy(img)
        label = torch.from_numpy(label)
        targets = label.view(1)
        return (img, targets)

Creating the custom dataset


In [6]:
!pip install ColossalAI deepspeed

     |████████████████████████████████| 234 kB 12.7 MB/s 
     |████████████████████████████████| 517 kB 38.5 MB/s 
     |████████████████████████████████| 124 kB 44.5 MB/s 
     |████████████████████████████████| 108 kB 49.0 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 18.2 MB 1.4 MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.5.8-py3-none-any.whl size=532153 sha256=88858780f9542c7783acb9289b8cb62d5f5472044bdb336c80927e523e1bba1a
  Stored in directory: /root/.cache/pip/wheels/8d/c6/82/cabd9a300c582a221591fd2c8c997e1f03f601e748aad44e4e
Successfully built deepspeed


In [7]:
import colossalai
from colossalai.engine import Engine, NoPipelineSchedule
from colossalai.trainer import Trainer
from colossalai.context import Config
import torch

Colossalai should be built with cuda extension to use the FP16 optimizer
Colossalai should be built with cuda extension to use the FP16 optimizer
apex is required for mixed precision training


First, we should initialize distributed environment. Though we just use single GPU in this example, we still need initialize distributed environment for compatibility. We just consider the simplest case here, so we just set the number of parallel processes to 1.

In [8]:
parallel_cfg = Config(dict(parallel=dict(
    data=dict(size=1),
    pipeline=dict(size=1),
    tensor=dict(size=1, mode=None),
)))
colossalai.init_dist(config=parallel_cfg,
          local_rank=0,
          world_size=1,
          host='127.0.0.1',
          port=8888,
          backend='nccl')

colossalai - torch.distributed.distributed_c10d - 2021-12-07 20:35:20,467 INFO: Added key: store_based_barrier_key:1 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-07 20:35:20,469 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 1 nodes.
colossalai - torch.distributed.distributed_c10d - 2021-12-07 20:35:20,470 INFO: Added key: store_based_barrier_key:2 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-07 20:35:20,476 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:2 with 1 nodes.
colossalai - torch.distributed.distributed_c10d - 2021-12-07 20:35:20,480 INFO: Added key: store_based_barrier_key:3 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-07 20:35:20,484 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:3 with 1 nodes.


process rank 0 is bound to device 0


Using transfer learning on inceptionV3 and creating a CNN model

In [9]:
import torch.nn as nn
import torchvision.models as models

class CNN(nn.Module):
    def __init__(self, train_CNN=False, num_classes=1):
        super(CNN, self).__init__()
        self.train_CNN = train_CNN
        self.inception = models.inception_v3(pretrained=True, aux_logits=False)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, images):
        features = self.inception(images)
        return self.sigmoid(self.dropout(self.relu(features))).squeeze(1)

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

Doing data augmentation and normalization

In [21]:
transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

In [17]:
learning_rate = 0.00001
train_CNN = False
batch_size = 1
shuffle = True
pin_memory = True
num_workers = 1

Creating Dataloader

In [22]:
dataset = CatsAndDogsDataset("/content/drive/MyDrive/Colab Notebooks/Train/train","train_csv.csv",transform=transform)

train_set, validation_set = torch.utils.data.random_split(dataset,[20000,5000])
train_loader = DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)

In [23]:
model = CNN().cuda()

for name, param in model.inception.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = train_CNN


Define a Loss function and optimizer. And then we use them to initialize Engine and Trainer. We provide various training / evaluating hooks. In this case, we just use the simplest hooks which can compute and print loss and accuracy.

In [24]:
import torch.optim as optim

# optimizer = optim.Adam(model.parameters(), lr=0.0001)
# criterion = nn.CrossEntropyLoss()

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
schedule = NoPipelineSchedule()

engine = Engine(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        lr_scheduler=None,
        schedule=schedule
    )
trainer = Trainer(engine=engine,
          hooks_cfg=[dict(type='LossHook'), dict(type='LogMetricByEpochHook'), dict(type='AccuracyHook')],
          verbose=True)

colossalai - rank_0 - 2021-12-07 22:17:24,328 WARNING: No gradient handler is set up, please make sure you do not need to all-reduce the gradients after a training step.
colossalai - rank_0 - 2021-12-07 22:17:24,331 INFO: build LogMetricByEpochHook for train, priority = 1
colossalai - rank_0 - 2021-12-07 22:17:24,333 INFO: build LossHook for train, priority = 10
colossalai - rank_0 - 2021-12-07 22:17:24,337 INFO: build AccuracyHook for train, priority = 10


Easily training on a GPU using ColossalAI which doesn't require much change from normal pytorch coding practices

In [25]:
num_epochs = 2
test_interval = 1
trainer.fit(
        train_dataloader=train_loader,
        test_dataloader=validation_loader,
        max_epochs=num_epochs,
        display_progress=True,
        test_interval=test_interval
    )

[Epoch 0 train]: 100%|██████████| 20000/20000 [12:12<00:00, 27.32it/s]
colossalai - rank_0 - 2021-12-07 22:29:54,685 INFO: Training - Epoch 1 - LogMetricByEpochHook: Loss = 0.69350
[Epoch 0 val]: 100%|██████████| 5000/5000 [02:47<00:00, 29.91it/s]
colossalai - rank_0 - 2021-12-07 22:32:41,957 INFO: Testing - Epoch 1 - LogMetricByEpochHook: Loss = 0.69314, Accuracy = 0.49960
[Epoch 1 train]: 100%|██████████| 20000/20000 [12:09<00:00, 27.42it/s]
colossalai - rank_0 - 2021-12-07 22:44:51,541 INFO: Training - Epoch 2 - LogMetricByEpochHook: Loss = 0.69332
[Epoch 1 val]: 100%|██████████| 5000/5000 [02:46<00:00, 29.95it/s]
colossalai - rank_0 - 2021-12-07 22:47:38,571 INFO: Testing - Epoch 2 - LogMetricByEpochHook: Loss = 0.69312, Accuracy = 0.49960


In [30]:
import torch.optim as optim

# optimizer = optim.Adam(model.parameters(), lr=0.0001)
# criterion = nn.CrossEntropyLoss()

# criterion = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
schedule = NoPipelineSchedule()

engine = Engine(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        lr_scheduler=None,
        schedule=schedule
    )
trainer = Trainer(engine=engine,
          hooks_cfg=[dict(type='LossHook'), dict(type='LogMetricByEpochHook'), dict(type='AccuracyHook')],
          verbose=True)

colossalai - rank_0 - 2021-12-07 23:08:57,510 WARNING: No gradient handler is set up, please make sure you do not need to all-reduce the gradients after a training step.
colossalai - rank_0 - 2021-12-07 23:08:57,514 INFO: build LogMetricByEpochHook for train, priority = 1
colossalai - rank_0 - 2021-12-07 23:08:57,515 INFO: build LossHook for train, priority = 10
colossalai - rank_0 - 2021-12-07 23:08:57,518 INFO: build AccuracyHook for train, priority = 10


In [31]:
num_epochs = 2
test_interval = 1
trainer.fit(
        train_dataloader=train_loader,
        test_dataloader=validation_loader,
        max_epochs=num_epochs,
        display_progress=True,
        test_interval=test_interval
    )

[Epoch 0 train]: 100%|██████████| 20000/20000 [12:07<00:00, 27.48it/s]
colossalai - rank_0 - 2021-12-07 23:21:08,658 INFO: Training - Epoch 1 - LogMetricByEpochHook: Loss = 0.69329
[Epoch 0 val]: 100%|██████████| 5000/5000 [02:47<00:00, 29.80it/s]
colossalai - rank_0 - 2021-12-07 23:23:56,564 INFO: Testing - Epoch 1 - LogMetricByEpochHook: Loss = 0.69313, Accuracy = 0.49960
[Epoch 1 train]: 100%|██████████| 20000/20000 [12:08<00:00, 27.47it/s]
colossalai - rank_0 - 2021-12-07 23:36:04,713 INFO: Training - Epoch 2 - LogMetricByEpochHook: Loss = 0.69329
[Epoch 1 val]: 100%|██████████| 5000/5000 [02:47<00:00, 29.88it/s]
colossalai - rank_0 - 2021-12-07 23:38:52,144 INFO: Testing - Epoch 2 - LogMetricByEpochHook: Loss = 0.69313, Accuracy = 0.49960
